<a href="https://colab.research.google.com/github/charlesreid1/deep-learning-genomics/blob/master/ray_tune_cnn1d_dna_transcription.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyperparameter Tuning with Ray, Tune, Keras, and Sklearn for Deep Learning Genomics

In this notebook, we use keras to assemble a 1D convolutional neural network for a deep learning application in genomics, and tune the network hyperparameters (parameters related to the network architecture and training algorithm) using sklearn.

In prior notebooks, we trained models and came to a better understanding of them using various metrics and plots. In this notebook, we re-use the figures and analysis from prior notebooks to help summarize a larger set of models, without as much of an explanation of how we arrived at those figures or how to interpret them. For more on that, see prior notebooks in the [charlesreid1/deep-learning-genomics](https://github.com/charlesreid1/deep-learning-genomics) repository on Github.

This notebook will use a framework called [tune](https://ray.readthedocs.io/en/latest/tune.html), part of the [ray](http://github.com/ray-project/ray) framework. 

## Import Libraries

In [53]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [54]:
import os
import math
import numpy as np
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
import pandas
import joblib

In [55]:
# sklearn
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.utils import class_weight
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [56]:
# keras
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, Model
from keras.layers import Flatten, Embedding, Dense, Dropout, Input, Concatenate
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.layers import LeakyReLU
import keras

In [57]:
!pip install ray &> /dev/null

In [70]:
import ray
from ray import tune

from ray.tune import Trainable
from ray.tune.util import pin_in_object_store, get_pinned_object

In [71]:
seed = 1729
numpy.random.seed(seed)

## Define Useful Keras Metrics

Before we get started assembling our model, we define a few useful metric functions to use with the model.

In [72]:
# via https://github.com/keras-team/keras/issues/6507#issuecomment-322857357

import keras.backend as K

def precision(y_true, y_pred):
    # Calculate the precision
    # clip ensures we're between 0 and 1
    # round ensures we're either 0 or 1
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # Calculate the recall
    # clip ensures we're between 0 and 1
    # round ensures we're either 0 or 1
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def fvalue(y_true, y_pred):
    # Calculate the F-value
    # If there are no true positives, fix the F score at 0 like sklearn.
    if K.sum(K.round(K.clip(y_true, 0, 1))) == 0:
        return 0
    p = precision(y_true,y_pred)
    r = recall(y_true,y_pred)
    fvalue = (2 * p * r)/(p + r + K.epsilon())
    return fvalue


## Load Dataset

In [73]:
!git clone https://github.com/deepchem/DeepLearningLifeSciences.git

fatal: destination path 'DeepLearningLifeSciences' already exists and is not an empty directory.


In [74]:
!ln -fs DeepLearningLifeSciences/Chapter06/{test*,train*,valid*} .

In [75]:
!ln -fs DeepLearningLifeSciences/Chapter06/chromatin.txt

In contrast to the prior example, which uses the already-provided splits of training, testing, and validation, we will load all of the data all at once into a single X and y pair and use sklearn to split the data into testing and training sets.

In [76]:
def load_all_data():
    
    # load chromatin accessibility data
    accessibility = {}
    for line in open('chromatin.txt','r'):
        fields = line.split()
        accessibility[fields[0]] = float(fields[1])
    
    # load training, validation, and testing sets
    for i,label in enumerate(['train','valid','test']):
        datadir = "%s_dataset"%(label)
        base_filename = "shard-0-%s.joblib"
        X_filename = os.path.join(datadir,base_filename%("X"))
        y_filename = os.path.join(datadir,base_filename%("y"))
        ids_filename = os.path.join(datadir,base_filename%("ids"))
        
        this_X = joblib.load(X_filename)
        this_y = joblib.load(y_filename)
        this_ids = joblib.load(ids_filename)
        this_chromatin = np.array([accessibility[k] for k in this_ids])
        
        # add X and chromatin data
        if i>0:
            X = np.concatenate([X,this_X])
            chromatin = np.concatenate([chromatin,this_chromatin])
            y = np.concatenate([y,this_y])
        else:
            X = this_X
            chromatin = this_chromatin
            y = this_y
        
    return [X,chromatin], y

# Parameter Tuning

In the sections that follow, we assemble a keras 1D CNN and tune parameters of the model.

## Using tune to tune hyperparameters

To use [tune](https://ray.readthedocs.io/en/latest/tune.html) to tune model hyperparameters, we will use the [tutorial on using tune with keras](https://github.com/ray-project/tutorial/blob/master/tune_exercises/Tutorial.ipynb) as a guide.

tune is installed as a component in [ray](https://github.com/ray-project/ray).

## Parameter Exploration Plan

At this point, we are ready to assemble the model. We should decide what parameters to adjust before assembling the model.

### Parameters that can be adjusted

* The number of stacked convolutional layers (3 in the original version of the problem)
* The number of filters in the convolution layers (16 in the original version of the problem)
* The convolution window size (10 in the original version of the problem)
* The dropout ratio for the dropout layers following the convolutional layers (0.5 originally)
* The sequence length we feed into the neural net (101 originally)
* Activation function used by the convolutional layers (RELU originally)
* Activation function used by final binary classifier output (sigmoid originally)
* Loss function (`binary_crossentropy` originally)
* Optimizer (`adam` originally)

### Parameters that we will adjust

Because models are expensive to train, and we must train one model from scratch for _each_ unique parameter combination, we want to start with parameters that we think will have the largest impact (the parameters the model's correctness is most sensitive to).

We list the three parameters that we will adjust below, with justification:

* **Number of stacked convolutional layers** - more layers means we have a higher potential to extract features at the given layer of resolution, so adding more layers can uncover more structure in the 1D DNA sequences

* **Number of filters in each convolutional layer** - more filters means more parts of the image are covered by more filters, making more likely that features will be extracted; adding more filters can reveal structures across more scales of the sequence; typically convolutional neural networks start with a small number of filters and cascade into more as the network gets deeper, so we explore a network that has increasing numbers of filters in each convolutional layer, and a network that has a fixed number of filters at each convolutional layer.

* **Convolution window size** - a larger convolution window size means we are able to examine more of the sequence _simultaneously_, meaning we are more likely to find non-local effects among features in our sequence

Why not simply crank up all the parameter values? Simpler networks have fewer parameters; as the network complexity increases, the need for data can explode. There is also a risk of overfitting the training data due to having too many parameters, and performing poorly on the general problem, so we want to try both simple and complex models.

### Parameter values

The initial hyperparameter study will search a three-parameter space, trying two values for each parameter. Our goal with this notebook is to demonstrate how to utilize hyperas, so we keep the search simple and limited in scope.

* Number of stacked convolutional layers: \[3, 4\]

* Number of convolutional filters at each layer (single filter size or doubling filter size):
    * 3 convolutional layers: \[16/16/16, 16/32/64\]
    * 4 convolutional layers: \[16/16/16/16, 16/32/64/128\]

* Convolution window size: \[10, 20\]

We are running $2^3 = 8$ cases:

| Case | No. Stacked Conv1D Layers | No. Conv1D Filters | Conv1D Window Size |
|------|---------------------------|--------------------|--------------------|
| 1*  | 3                           | 16/16/16     | 10                |
| 2    | 3                           | 16/16/16     | 25                |
| 3    | 3                           | 16/32/64     | 10                |
| 4    | 3                           | 16/32/64     | 25                |
| 5    | 4                           | 16/16/16/16     | 10                |
| 6    | 4                           | 16/16/16/16     | 20                |
| 7    | 4                           | 16/32/64/128    | 10                |
| 8    | 4                           | 16/32/64/128    | 20                |


In [77]:
import itertools
from pprint import pprint

hyperparam_combos = list(itertools.product([3,4],["16/16/...","16/32/..."],[10,20]))
pprint(hyperparam_combos)

[(3, '16/16/...', 10),
 (3, '16/16/...', 20),
 (3, '16/32/...', 10),
 (3, '16/32/...', 20),
 (4, '16/16/...', 10),
 (4, '16/16/...', 20),
 (4, '16/32/...', 10),
 (4, '16/32/...', 20)]


To create a model that hyperas can call with the different parameter combinations, we will insert special calls to hyperas to tell it which parameter values to insert at different points in the function that creates the keras model. This will allow hyperas to create different models with different parameters or architectures, and to bake complex logic into the variations.

## Defining a Trainable

Now we are ready to create a function to help automate the hyperparameter search.

The hyperparameter search will take place as a series of trials. Each trial runs a Python function called a trainable.

The trainable function must:

- pass in a tune_reporter object as a parameter: `def my_trainable(config, tune_reporter)`
- train the model using the given parameters
- report metrics to tune (so it can keep track of the model's performance over time)

### Integrating with Keras via Keras Callback

To integrate tune with keras, you can use a custom keras callback class, `TuneCallback`, which will report metrics.

In [78]:
class TuneCallback(keras.callbacks.Callback):
    """Custom Callback for Tune."""

    def __init__(self, reporter):
        super(TuneCallback, self).__init__()
        self.reporter = reporter
        self.metrics = ['acc','precision','recall','fvalue']
        self.most_important_metric = 'precision'
        
        # Top score for each metric
        self.top = {}
        for m in self.metrics:
            self.top[m] = -1
        
        # Last 10 scores for each metric
        self.last_10 = {}
        for m in self.metrics:
            self.last_10[m] = []
        
    def on_batch_end(self, batch, logs={}):
        """Reports the last result"""
        curr = logs[self.most_important_metric]
        top = self.top[self.most_important_metric]
        
        if curr > top:
            self.model.save_weights('weights_tune_tmp.h5')
            os.rename('weights_tune_tmp.h5','weights_tune.h5')
        
        reporter_kwargs = {}
        
        # For each metric we've defined (top of notebook),
        # we want to keep track of the moving average
        for m in self.metrics:
            
            # Drop oldest value
            if len(self.last_10[m]) >= 5:
                self.last_10[m] = self.last_10[m][1:]
                
            # Append newest value
            self.last_10[m] = self.last_10[m] + [logs[m]]
            
            # Get name for mean metric
            mean_metric = 'mean_' + m
            
            # Report mean of last 10 values of this metric
            reporter_kwargs[mean_metric] = np.mean(self.last_10[m])
        
        reporter_kwargs['checkpoint'] = 'weights_tune.h5'
        
        self.reporter(**reporter_kwargs)

### The Trainable Function

The trainable function must:

- pass in a tune_reporter object as a parameter: `def my_trainable(config, tune_reporter)`
- train the model using the given parameters
- report metrics to tune (so it can keep track of the model's performance over time)

We split the trainable into two functions: 

* The first function `keras_chromastin_model()` assembles and returns uncompiled keras neural network with user-specified architecture parameters

* The second function `tune_cnn_trainable(config, tune_reporter)` is the trainable function that is actually called by tune

In [79]:
# This function is called from the trainable function.
# The trainable function gets the hyperparameters specified
# in the input dictionary (and picked by tune), and uses them
# to call this function to assemble the keras CNN model.

def keras_chromatin_model(n_convolutional_layers, n_convolutional_filters, convolution_window, seq_length, n_features):
    """
    This function takes network architecture parameters as inputs,
    and returns an uncompiled keras 1D convolutional neural network
    with the specified architecture.
    """
    # ----------------------------
    # Sequence branch of network
    # (1D DNA sequence)
    
    # Input
    seq_in = Input(shape=(seq_length,n_features))
    
    # Fencepost pattern
    seq = seq_in
    
    for i in range(n_convolutional_layers):
        seq = Conv1D(n_convolutional_filters[i], 
                     convolution_window,
                     activation='relu', 
                     padding='same',
                     kernel_initializer='normal')(seq)
        seq = Dropout(0.5)(seq)
    
    # Flatten to 1D
    seq = Flatten()(seq)
    
    # Assemble the sequential branch of network
    seq = keras.Model(inputs=seq_in, outputs=seq)
    
    # ---------------------------
    # Chromatin branch of network
    
    # Input
    chrom_input = Input(shape=(1,))

    # ---------------------------
    # Combine networks
    fin = keras.layers.concatenate([seq.output, chrom_input])
    fin = Dense(1,
                activation='sigmoid', 
                kernel_initializer='normal')(fin)
    chrom_model = keras.Model(inputs=[seq.input,chrom_input], 
                              outputs=fin)

    return chrom_model

### Efficiently Handling Large Data Sets

To efficiently handle a large data set, we can load it once and pin it in memory using the tune object store. This is particularly convenient when the data lives in a file, as ray workers may not all have access to files on disk.

We pin objects using tune's `pin_object_in_store()` utility function, then retrieve it using the `get_pinned_object()` utility function.

In [100]:
ray.shutdown(); ray.init(ignore_reinit_error=True)

[X_temp,chromatin_temp], y_temp = load_all_data()
X_ = pin_in_object_store(X_temp)
chromatin_ = pin_in_object_store(chromatin_temp)
y_ = pin_in_object_store(y_temp)

2019-06-10 17:00:26,297	INFO node.py:497 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-10_17-00-26_296667_85233/logs.
2019-06-10 17:00:26,411	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:61814 to respond...
2019-06-10 17:00:26,539	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:47608 to respond...
2019-06-10 17:00:26,544	INFO services.py:806 -- Starting Redis shard with 1.72 GB max memory.
2019-06-10 17:00:26,592	INFO node.py:511 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-06-10_17-00-26_296667_85233/logs.
2019-06-10 17:00:26,598	INFO services.py:1441 -- Starting the Plasma object store with 2.58 GB memory using /tmp.


### Define Trainable Function

The next step is to define the trainable function - this is the function that is repeatedly called by tune during the process of optimizing and searching hyperparamter space.

It takes two arguments as input - a config dictionary (containing the hyperparameter values that tune is passing to the model), and a tune reporter, which we use to connect the function caller (tune) with the function being called (the `train()` method of our keras model).

In [101]:
def tune_cnn_trainable(config, tune_reporter):
    """
    This function is a trainable function that is used by tune
    to run the various hyperparameter configurations.
    
    This function should:
    
    - pass in a tune_reporter object as a parameter: def my_trainable(config, tune_reporter)
    - train the model using the given parameters
    - report metrics to tune (so it can keep track of the model's performance over time)
    
    The input data X and y should come from a generator.
    This trainable function should take care of creating
    the model, creating the input data, using the input data
    to train the model, and reporting metrics back.
    """
    #############################################
    # Problem-specific parameters
    
    # length of k-mer over which we're convoluting
    seq_length = 101
    
    # length of DNA alphabet
    n_features = 4
    
    # cross-validation settings
    n_fold = 2
    
    # colors to associate with each fold
    colors = ['dusty purple','dusty green']
    fold_colors = [sns.xkcd_rgb[j] for j in colors]
    
    # network settings
    n_epochs = 5
    batch_size = 2048
    
    if tune_reporter is None:
        # it's a smoke test
        n_epochs = 1
    
    
    ############################################
    # Hyperparameter values come from config
    
    n_convolutional_layers = config.get("n_convolutional_layers",3)
    convolutional_filters_mode = config.get("convolutional_filters_mode",16)
    convolutional_window = config.get("convolutional_window",10)
    
    unique_param_string = "layers%d_filtermode%d_window%d"%(
            n_convolutional_layers,
            convolutional_filters_mode,
            convolutional_window
    )
    
    
    #############################################
    # Construct network
    
    # if mode = 16, use all 16s for every layer
    # if mode = 32, double each layer until 64
    n_convolutional_filters = None
    if convolutional_filters_mode==16:
        if n_convolutional_layers==3:
            n_convolutional_filters = [16,16,16]
        elif n_convolutional_layers==4:
            n_convolutional_filters = [16,16,16,16]
            
    elif convolutional_filters_mode==32:
        if n_convolutional_layers==3:
            n_convolutional_filters = [16,32,64]
        elif n_convolutional_filters==4:
            n_convolutional_filters = [16,32,64,64]

    if n_convolutional_filters==None:
        msg = "Error: invalid number of filters per layer or number of layers, could not set number of filters for layers."
        raise Exception(msg)

    chrom_model = keras_chromatin_model(n_convolutional_layers, 
                                        n_convolutional_filters, 
                                        convolutional_window, 
                                        seq_length, 
                                        n_features)
    
    
    #############################################
    # Input data
    
    # Here, we need to load X and y
    X = get_pinned_object(X_)
    chromatin = get_pinned_object(chromatin_)
    y = get_pinned_object(y_)

    # stratified = ratios of classes are preserved in each fold
    shuffle = StratifiedShuffleSplit(n_splits=n_fold,
                                     train_size = 0.7,
                                     test_size = 0.3,
                                     random_state = seed)
    
    # ------------
    # Begin cross-validation process
    #
    # Iterate over each of the k-fold training/testing splits
    # and create and train a model from scratch. Save the
    # training history, then return the model and its history.
    
    for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):
        
        #############################################
        # Model training
        
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        chrom_train, chrom_test = chromatin[train_ix], chromatin[test_ix]

        # ---------------------------
        # Assemble class weights 
        # from the training data only
        # (no peeking at the test data!)
        classes = np.unique(y_train)
        labels = np.squeeze(y_train)
        weights = class_weight.compute_class_weight('balanced',classes,labels)
        class_weights = {}
        for c,w in zip(classes,weights):
            class_weights[c] = w
        
        # Compile the model
        chrom_model.compile(loss='binary_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy',
                               precision,
                               recall,
                               fvalue])
        
        if tune_reporter is not None:
            # This is for tune
            callbacks = [TuneCallback(tune_reporter)]
            verb = 0
        else:
            callbacks = []
            verb = 1
        
        # Fit to data
        hist = chrom_model.fit([X_train,chrom_train], y_train,
                               batch_size = batch_size,
                               epochs = n_epochs,
                               class_weight = class_weights,
                               verbose = verb,
                               callbacks = callbacks,
                               validation_data = ([X_test,chrom_test], y_test))
        
        #histories.append(hist)
        #models.append(chrom_model)

### Test the Reporter

In [102]:
class GoodError(Exception):
    pass

def test_reporter(tune_cnn_trainable):
    def mock_reporter(**kwargs):
        #assert "mean_acc" in kwargs, "Did not report mean_accuracy metric"
        #assert "checkpoint" in kwargs, "Accidentally removed `checkpoint`?"
        print("The tune trainable has the following keyword args:")
        print(kwargs.keys())
        print()
        raise GoodError("This works.")
    try:
        tune_cnn_trainable({}, mock_reporter)
    except TypeError as e:
        print("Forgot to modify function signature?")
        raise e
    except GoodError:
        print("Works!")
        return 1
    raise Exception("Didn't call reporter...")

assert test_reporter(tune_cnn_trainable)

The tune trainable has the following keyword args:
dict_keys(['mean_acc', 'mean_precision', 'mean_recall', 'mean_fvalue', 'checkpoint'])

Works!


In [103]:
# Smoke test: make sure that when we call the above function,
# we successfully create and train a model.
# This will train k models (because k-fold cross validation).

if False:
    
    tune_cnn_trainable({
            'n_convolutional_layers':3,
            'convolutional_filters_mode':16,
            'convolutional_window':10
        },None)

In [104]:
stop={"training_iteration": 10}

In [105]:
space = {
    'n_convolutional_layers': tune.grid_search([3]),
    'convolutional_filters_mode': tune.grid_search([16,32]),
    'convolutional_window': tune.grid_search([10,25])
}

In [106]:
space_slim = {
    'n_convolutional_layers': tune.grid_search([3]),
    'convolutional_filters_mode': tune.grid_search([16]),
    'convolutional_window': tune.grid_search([10])
}

In [107]:
# Start with slim parameter space exploration (one parameter set)
slim_trials = tune.run(
    tune_cnn_trainable,
    stop=stop, 
    config=space_slim,
    resources_per_trial={"cpu": 1},
    verbose=1
)

2019-06-10 17:01:02,260	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-06-10 17:01:02,262	INFO tune.py:223 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.4/8.6 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.4/8.6 GB
Result logdir: /Users/charles/ray_results/tune_cnn_trainable
Number of trials: 1 ({'RUNNING': 1})
RUNNING trials:
 - tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3:	RUNNING

(pid=85949) Using TensorFlow backend.
(pid=85949) WARNING:tensorflow:From /Users/charles/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/tensorflow/python/framework/op_def_library.py:263: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
(pid=85949) Instructions for updating:
(pid=85949) Colocations handled automatically by placer.
(pid=85949) WARNING:tensorflow:From /Users/charles/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/keras

2019-06-10 17:01:16,650	INFO ray_trial_executor.py:180 -- Destroying actor for trial tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.7/8.6 GB
Result logdir: /Users/charles/ray_results/tune_cnn_trainable
Number of trials: 1 ({'TERMINATED': 1})
TERMINATED trials:
 - tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3:	TERMINATED, [1 CPUs, 0 GPUs], [pid=85949], 9 s, 10 iter



In [108]:
print(type(slim_trials[0]))
print()
print(dir(slim_trials[0]))
print()
print(slim_trials[0].history)
print()
print(slim_trials[0].last_result)

<class 'ray.tune.trial.Trial'>

['ERROR', 'PAUSED', 'PENDING', 'RUNNING', 'TERMINATED', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_checkpoint', '_cmp_greater', '_get_trainable_cls', '_nonjson_fields', '_registration_check', '_status_string', 'best_checkpoint_attr_value', 'best_result', 'checkpoint_at_end', 'checkpoint_freq', 'checkpoint_score_attr', 'clear_checkpoint', 'close_logger', 'compare_checkpoints', 'config', 'custom_trial_name', 'error_file', 'experiment_tag', 'export_formats', 'extra_arg', 'generate_id', 'has_checkpoint', 'history', 'init_logger', 'is_finished', 'keep_checkpoints_num', 'last_debug', 'last_result', 'last_update_time', 'local_dir', 

In [109]:
# Now explore the whole parameter space
full_trials = tune.run(
    tune_cnn_trainable,
    stop=stop, 
    config=space,
    resources_per_trial={"cpu": 1},
    verbose=1
)

2019-06-10 17:01:47,540	INFO tune.py:60 -- Tip: to resume incomplete experiments, pass resume='prompt' or resume=True to run()
2019-06-10 17:01:47,541	INFO tune.py:223 -- Starting a new experiment.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.6/8.6 GB

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 1/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.6/8.6 GB
Result logdir: /Users/charles/ray_results/tune_cnn_trainable
Number of trials: 4 ({'RUNNING': 1, 'PENDING': 3})
PENDING trials:
 - tune_cnn_trainable_1_convolutional_filters_mode=32,convolutional_window=10,n_convolutional_layers=3:	PENDING
 - tune_cnn_trainable_2_convolutional_filters_mode=16,convolutional_window=25,n_convolutional_layers=3:	PENDING
 - tune_cnn_trainable_3_convolutional_filters_mode=32,convolutional_window=25,n_convolutional_layers=3:	PENDING
RUNNING trials:
 - tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3:	RUNNING

(pid=85952) Using TensorFlow backend.
(pid=85950) Using TensorFlow backend.
(pid=85951) Using TensorFlow backend.
(pid=85982) Using TensorFlow backend.
(pid=8

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.8/8.6 GB
Result logdir: /Users/charles/ray_results/tune_cnn_trainable
Number of trials: 4 ({'RUNNING': 4})
RUNNING trials:
 - tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85950], 20 s, 7 iter
 - tune_cnn_trainable_1_convolutional_filters_mode=32,convolutional_window=10,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85951], 21 s, 3 iter
 - tune_cnn_trainable_2_convolutional_filters_mode=16,convolutional_window=25,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85952], 20 s, 3 iter
 - tune_cnn_trainable_3_convolutional_filters_mode=32,convolutional_window=25,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85982], 19 s, 1 iter



2019-06-10 17:02:22,144	INFO ray_trial_executor.py:180 -- Destroying actor for trial tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


== Status ==
Using FIFO scheduling algorithm.
Resources requested: 4/4 CPUs, 0/0 GPUs
Memory usage on this node: 7.0/8.6 GB
Result logdir: /Users/charles/ray_results/tune_cnn_trainable
Number of trials: 4 ({'RUNNING': 4})
RUNNING trials:
 - tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85950], 25 s, 9 iter
 - tune_cnn_trainable_1_convolutional_filters_mode=32,convolutional_window=10,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85951], 26 s, 4 iter
 - tune_cnn_trainable_2_convolutional_filters_mode=16,convolutional_window=25,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85952], 25 s, 4 iter
 - tune_cnn_trainable_3_convolutional_filters_mode=32,convolutional_window=25,n_convolutional_layers=3:	RUNNING, [1 CPUs, 0 GPUs], [pid=85982], 19 s, 1 iter

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/4 CPUs, 0/0 GPUs
Memory usage on this node: 6.7/8.6 GB
Result lo

Exception in thread ray_print_logs:
Traceback (most recent call last):
  File "/Users/charles/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/redis/connection.py", line 185, in _read_from_socket
    raise socket.error(SERVER_CLOSED_CONNECTION_ERROR)
OSError: Connection closed by server.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/charles/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/charles/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/charles/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/ray/worker.py", line 1554, in print_logs
    msg = pubsub_client.get_message()
  File "/Users/charles/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/redis/client.py", line 3135, in get_message
    response = self.

KeyboardInterrupt: 

In [ ]:
t = full_trials[0]

print(t)
print()
print(type(t))
print()
print(dir(t))
print()
print(t.logdir)

In [ ]:
!ls "/Users/charles/ray_results/tune_cnn_trainable/tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3_2019-06-09_14-45-29_pk90zjj"

In [ ]:
!cat "/Users/charles/ray_results/tune_cnn_trainable/tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3_2019-06-09_14-45-29_pk90zjj/result.json"

In [ ]:
#####################################
# LOAD HISTORY FOR ONE RUN
#
# This is the dumb way to do it.
# The smart way is to utuilize
# the built-in functionality that
# tune provides to you.
# 
# https://ray.readthedocs.io/en/latest/tune-usage.html#logging-analyzing-and-visualizing-results
#
#####################################

import json
import pandas as pd
from pprint import pprint

history = []
with open("/Users/charles/ray_results/tune_cnn_trainable/tune_cnn_trainable_0_convolutional_filters_mode=16,convolutional_window=10,n_convolutional_layers=3_2019-06-09_14-45-29_pk90zjj/result.json","r") as f:
    for line in f.readlines():
        # One dictionary per line
        d = json.loads(line)
        
        # Flatten the (nested) config dictionary
        temp = d['config']
        del d['config']
        for k in temp.keys():
            d[k] = temp[k]
        
        #pprint(d)
        history.append(d)

df = pd.DataFrame(history)

fig, axes = plt.subplots(1,2,figsize=(12,4))

df['mean_acc'].plot(ax=axes[0])
df['mean_precision'].plot(ax=axes[1])

axes[0].set_title('Mean Accuracy')
axes[0].set_xlabel('Training Iteration')
axes[0].set_ylabel('Accuracy')

axes[1].set_title('Mean Precision')
axes[1].set_xlabel('Training Iteration')
axes[1].set_ylabel('Precision')

plt.tight_layout()

In [ ]:
#######################
# THE EASY WAY
#######################

from ray.tune.analysis import ExperimentAnalysis

ea = ExperimentAnalysis("~/ray_results/my_experiment")
trials_dataframe = ea.dataframe()

This is currently our stopping point.


...



...



...


...

In [ ]:
#############################################
# Post-process metrics

'''
We have multiple metrics for each model:
- accuracy
- precision
- recall
- f1 score
- all of above at each epoch

Once we use each model to evaluate y_test, we can get:
- confusion matrix
- construct roc with confidence interval
- roc_auc_score

We can aggregate these metrics many ways, 
but for now we stick to using roc_auc_score.
hyperas will use that to pick the best model.

We should make the plots and save them to 
image files, then display the image files in
notebook cells below when we analyze the results
of the hyperparamter study.
'''
# ---------------------
# Compute aggregate statistics
# for each metric

loss      = [h.history['val_loss'] for h in histories]
loss_mean = np.mean(loss)
loss_std  = np.std(loss)

acc       = [h.history['val_acc'] for h in histories]
acc_mean  = np.mean(acc)
acc_std   = np.std(acc)

prec      = [h.history['val_pprecision'] for h in histories]
prec_mean = np.mean(prec)
prec_std  = np.std(prec)

rec       = [h.history['val_recall'] for h in histories]
rec_mean  = np.mean(rec)
rec_std   = np.std(rec)



# ---------------------
# Define functions to plot metric histories

def loss_rate_plot(histories, ax, label='', legend=False):

    for ih, hist in enumerate(histories):

        ax.plot(hist.history['loss'], 
                label='Train Loss (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='--')

        ax.plot(hist.history['val_loss'], 
                label='Test Loss (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='-')

    if label=='':
        ax.set_title("Loss", size=14)
    else:
        ax.set_title("Loss (%s)"%(label), size=14)

    ax.set_ylabel('Loss')
    ax.set_xlabel('Training epoch')

    if legend:
        ax.legend(loc='upper right')

def accuracy_rate_plot(histories, ax, label='', legend=False):

    for ih, hist in enumerate(histories):

        ax.plot([j*100 for j in hist.history['acc']],
                label='Train Acc (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='--')

        ax.plot([j*100 for j in hist.history['val_acc']],
                label='Test Acc (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='-')

    if label=='':
        ax.set_title("Accuracy", size=14)
    else:
        ax.set_title("Accuracy (%s)"%(label), size=14)

    ax.set_ylabel('Accuracy %')
    ax.set_xlabel('Training epoch')

    if legend:
        ax.legend(loc='lower right')

def precision_rate_plot(histories, ax, label='', legend=False):

    for ih, hist in enumerate(histories):

        ax.plot([j*100 for j in hist.history['precision']],
                label='Train Recall (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='--')

        ax.plot([j*100 for j in hist.history['val_precision']],
                label='Test Recall (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='-')

    if label=='':
        ax.set_title("Precision", size=14)
    else:
        ax.set_title("Precision (%s)"%(label), size=14)

    ax.set_ylabel('Precision %')
    ax.set_xlabel('Training epoch')

    if legend:
        ax.legend(loc='lower right')

def recall_rate_plot(histories, ax, label='', legend=False):

    for ih, hist in enumerate(histories):

        ax.plot([j*100 for j in hist.history['recall']],
                label='Train Recall (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='--')

        ax.plot([j*100 for j in hist.history['val_recall']],
                label='Test Recall (%d)'%(ih+1),
                color=fold_colors[ih],
                linestyle='--')

    if label=='':
        ax.set_title("Recall", size=14)
    else:
        ax.set_title("Recall (%s)"%(label), size=14)

    ax.set_ylabel('Recall %')
    ax.set_xlabel('Training epoch')

    if legend:
        ax.legend(loc='lower right')


# ---------------------
# Define functions to make confusion matrix plot

def plot_confusion_matrix(ax, y_true, y_pred, classes,
                          title=None,
                          cmap=plt.cm.Blues):

    if title is None:
        title = "Confusion Matrix"

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot confusion matrix
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)

    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), 
             rotation=45,
             ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else ','
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", 
                    va="center",
                    color="white" if cm[i, j] > thresh else "black")

# ---------------------
# Define functions to plot ROC curves

def plot_roc_confidence(models, ax, X, chromatin, y, shuffle):
    """Plot the ROC curve with confidence interval on ax.
    Return the mean and std of the ROC AUC, computed over
    each of the k folds from cross-validation.
    """
    tprs = []
    aucs = []
    mean_fpr = np.linspace(0,1,100)

    for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):

        print("Working on fold %d..."%(ifold+1))

        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        chrom_train, chrom_test = chromatin[train_ix], chromatin[test_ix]

        model = models[ifold]

        y_test_pred = model.predict([X_test,chrom_test]).ravel()

        fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)

        tprs.append(interp(mean_fpr,fpr,tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr,tpr)
        aucs.append(roc_auc)

        # Draw every ROC curve, lightly
        ax.plot(fpr,tpr, alpha=0.3)

mean_tpr = np.mean(tprs,axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)

# Draw the mean curve
ax.plot(mean_fpr, mean_tpr, 
        color='b', alpha=0.8)

# Now fill in the area between +/- std
std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, 
                color='grey', alpha=0.2)

# Diag for ref
ax.plot([0,1],[0,1],'--k',lw=2)

ax.set_title('ROC Curve, 1D CNN With Chromatin')

return mean_auc, std_auc


####################################
# Make plots

# ---------------------
# Plot metric histories for all the folds
# and save the plot to a file

# 4 subplots for 4 metrics
fig, [[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2, figsize=(8,6))

loss_rate_plot(fithists[i], ax1, legend=True)
accuracy_rate_plot(fithists[i], ax2, legend=True)
precision_rate_plot(fithists[i], ax3, legend=True)
recall_rate_plot(fithists[i], ax4, legend=True)

metric_histories_img = 'MetricHistories_%s.png'%(unique_param_string)
print("Saving metric histories to %s"%(metric_histories_img))
fig.savefig(metric_histories_img)


# ---------------------
# Plot confusion matrix 
# for all the folds for this model.
# Save plots to files.

cm_fig, cm_axes = plt.subplots(nrows=len(histories),ncols=1)
roc_fig, roc_axes = plt.subplots()

for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):
    hist = histories[ifold]
    model = models[ifold]

    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    chrom_train, chrom_test = chromatin[train_ix], chromatin[test_ix]

    y_test_pred = model.predict([X_test,chrom_test])
    y_test_pred = np.round(y_test_pred)

    # Confusion matrix plot
    cm_title = "Confusion Matrix (Fold %d)"%(ifold+1)
    cm_ax = cm_axes[ifold]        
    plot_confusion_matrix(cm_ax,y_test,y_test_pred,title=cm_title)

cm_img = 'ConfusionMatrix_%s.png'%(unique_param_string)
print("Saving confusion matrices to %s"%(cm_img))
cm_fig.savefig(cm_img)

# ---------------------
# Plot ROC curve (and get ROC AUC)
# for all the folds for this model.
# Save plots to files.

roc_fig, roc_ax = plt.subplots(1,1, figsize=(8,6))

mean_auc, std_auc = plot_roc_confidence(models, roc_ax, X, chromatin, y, shuffle)

roc_img = 'ROCConfidence_%s.png'%(unique_param_string)
print('Saving ROC confidence curve to %s'%(roc_img))
roc_fig.savefig(roc_img)

print("Area under ROC curve:")
print("\tMean: %0.4f"%(mean_auc))
print("\tStd: %0.4f"%(std_auc))

# which model are we supposed to return?
# can we return a list? 
# can we create an aggregate model that wraps all 3? 

return {'loss': mean_auc, 'status': STATUS_OK, 'model': models}

## Performing Cross Validation Manually

To perform cross validation and incorporate sample weights for imbalanced classes (many more negative examples than positive examples), we can't use weights with sklearn directly, so we do cross-validation manually.

The [StratifiedKFold](https://sklearn.org/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold) object can give us a set of k testing/training sets by using the `split()` method, which returns an iterator with the training/testing indices. This allows us to assemble all inputs (X and y, weights, labels, chromatin accessibility, etc.).

The [StratifiedShuffleSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html#sklearn.model_selection.StratifiedShuffleSplit) object also provides a split method.

In [ ]:
n_epochs = 100
n_fold = 3
include_chromatin_data = True

# we can use either of these,
# but we'll opt for shuffle

kfold = StratifiedKFold(n_splits=n_fold, 
                        shuffle=True, 
                        random_state=seed)

shuffle = StratifiedShuffleSplit(n_splits=n_fold,
                                 train_size = 0.7,
                                 test_size = 0.3,
                                 random_state = seed)

models = []
fithists = []

for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):
    
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    w_train, w_test = np.squeeze(w[train_ix]), np.squeeze(w[test_ix])
    chrom_train, chrom_test = np.squeeze(chromatin[train_ix]), np.squeeze(chromatin[test_ix])
    
    print("Training on fold %d..."%(ifold+1))
    
    
    # if we use the chromatin model, we need a list of inputs
    
    if include_chromatin_data:
        model = create_chromatin()
        hist = model.fit([X_train,chrom_train], y_train,
                         sample_weight = w_train,
                         batch_size = 1000,
                         epochs = n_epochs,
                         verbose = 0,
                         validation_data=([X_test,chrom_test],y_test,w_test))
    else:
        model = create_baseline()
        hist = model.fit(X_train, y_train,
                         sample_weight = w_train,
                         batch_size = 1000,
                         epochs = n_epochs,
                         verbose = 0,
                         validation_data=(X_test,y_test,w_test))

    models.append(model)
    fithists.append(hist)
    
    print("Done")
    

In [ ]:
print("Model results (validation):")
print("\n")
print("Loss (Mean):      %0.4f"%(np.mean([h.history['val_loss'] for h in fithists])))
print("Loss (Std):       %0.4f"%(np.std([h.history['val_loss'] for h in fithists])))
print("\n")
print("Accuracy (Mean):  %0.2f%%"%(100*np.mean([h.history['val_acc'] for h in fithists])))
print("Accuracy (Std):   %0.2f%%"%(100*np.std([h.history['val_acc'] for h in fithists])))
print("\n")
print("Precision (Mean): %0.2f%%"%(100*np.mean([h.history['val_precision'] for h in fithists])))
print("Precision (Std):  %0.2f%%"%(100*np.std([h.history['val_precision'] for h in fithists])))
print("\n")
print("Recall (Mean):    %0.2f%%"%(100*np.mean([h.history['val_recall'] for h in fithists])))
print("Recall (Std):     %0.2f%%"%(100*np.std([h.history['val_recall'] for h in fithists])))

In [ ]:
def loss_rate_plot(hist, ax, label='',legend=False):
    ax.plot(hist.history['loss'])
    ax.plot(hist.history['val_loss'])
    if label=='':
        ax.set_title("Loss Rate", size=14)
    else:
        ax.set_title("Loss Rate (%s)"%(label), size=14)
    ax.set_ylabel('Loss')
    ax.set_xlabel('Training interations')
    if legend:
        ax.legend(['Training', 'Validation'], loc='upper right')

def accuracy_rate_plot(hist,ax,label='',legend=False):
    ax.plot([j*100 for j in hist.history['acc']])
    ax.plot([j*100 for j in hist.history['val_acc']])
    if label=='':
        ax.set_title("Accuracy", size=14)
    else:
        ax.set_title("Accuracy (%s)"%(label), size=14)
    ax.set_ylabel('Accuracy %')
    ax.set_xlabel('Training iterations')
    if legend:
        ax.legend(['Training','Validation'], loc='lower right')

def precision_rate_plot(hist,ax,label='',legend=False):
    ax.plot([j*100 for j in hist.history['precision']])
    ax.plot([j*100 for j in hist.history['val_precision']])
    if label=='':
        ax.set_title("Precision", size=14)
    else:
        ax.set_title("Precision (%s)"%(label), size=14)
    ax.set_ylabel('Precision %')
    ax.set_xlabel('Training iterations')
    if legend:
        ax.legend(['Training','Validation'], loc='lower right')

def recall_rate_plot(hist,ax,label='',legend=False):
    ax.plot([j*100 for j in hist.history['recall']])
    ax.plot([j*100 for j in hist.history['val_recall']])
    if label=='':
        ax.set_title("Recall", size=14)
    else:
        ax.set_title("Recall (%s)"%(label), size=14)
    ax.set_ylabel('Recall %')
    ax.set_xlabel('Training iterations')
    if legend:
        ax.legend(['Training','Validation'], loc='lower right')

In [ ]:
for i in range(shuffle.get_n_splits()):
    
    fig, [[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2, figsize=(8,6))

    loss_rate_plot(fithists[i], ax1, legend=True, label="Fold %d"%(i+1))
    accuracy_rate_plot(fithists[i], ax2, label="Fold %d"%(i+1))
    precision_rate_plot(fithists[i], ax3, label="Fold %d"%(i+1))
    recall_rate_plot(fithists[i], ax4, label="Fold %d"%(i+1))

    fig.tight_layout()
    
plt.show()

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else ','
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):

    print("---------------")
    print("Fold %d:"%(ifold+1))
    
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    w_train, w_test = np.squeeze(w[train_ix]), np.squeeze(w[test_ix])
    chrom_train, chrom_test = chromatin[train_ix], chromatin[test_ix]
    
    model = models[ifold]
    
    if include_chromatin_data:
        y_test_pred = model.predict([X_test,chrom_test])
    else:
        y_test_pred = model.predict(X_test)
        
    y_test_pred = np.round(y_test_pred)

    ax = plot_confusion_matrix(y_test, y_test_pred, ['0','1'], title="Confusion Matrix - Fold %d"%(ifold+1))


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [ ]:
def plot_roc_curve(y_true, y_pred, weights):
    fpr, tpr, _ = roc_curve(y_true, y_pred)#, 
                            #sample_weight = weights)
    
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc_score(y_true, y_pred))
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.plot([0,1],[0,1],'k--')
    ax.set_title("ROC Curve")
    plt.legend(loc='lower right')
    plt.show()
    
    fig.tight_layout()
    return ax

Finally, we compute the ROC curve for all k models from the k-fold cross-validation and use those curves to assemble the mean and variance of the ROC curve at every point. See [ROC with cross validation example](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html) in the sklearn documentation.

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html

from scipy import interp

fig, ax = plt.subplots(1,1,figsize=(8,8))

tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)

for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):

    print("Working on fold %d..."%(ifold+1))
    
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    w_train, w_test = np.squeeze(w[train_ix]), np.squeeze(w[test_ix])
    chrom_train, chrom_test = chromatin[train_ix], chromatin[test_ix]
    
    model = models[ifold]
    
    if include_chromatin_data:
        y_test_pred = model.predict([X_test,chrom_test]).ravel()
    else:
        y_test_pred = model.predict(X_test).ravel()
        
    fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
    tprs.append(interp(mean_fpr,fpr,tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr,tpr)
    aucs.append(roc_auc)
    ax.plot(fpr,tpr, alpha=0.3)


mean_tpr = np.mean(tprs,axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b', alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2)

ax.plot([0,1],[0,1],'--k',lw=2)

ax.set_title('ROC Curve, 1D CNN With Chromatin')

print("Area under curve:")
print("Mean: %0.4f"%(mean_auc))
print("Std: %0.4f"%(std_auc))

plt.show()

## Class Weights

Next we try replacing the weights that came with the data set with class weights. Let's review the imbalance in our data:

In [ ]:
neg = np.sum(y==0)
pos = np.sum(y==1)
print("Negative examples: %d"%(neg))
print("Positive examples: %d"%(pos))
print("Percent positive: %0.2f%%"%(100*(pos/(pos+neg))))

We can use the `class_weight` module from sklearn to automatically compute class weights, which can be passed to keras when we fit our model to data.

In [ ]:
from sklearn.utils import class_weight

classes = np.unique(y_train)
labels = np.squeeze(y_train)

weights = class_weight.compute_class_weight('balanced',classes,labels)

class_weights = {}
for c,w in zip(classes,weights):
    class_weights[c] = w
print(class_weights)

Now we re-fit the model to data, this time using the class weights from sklearn instead of the weights provided in the data files:

In [ ]:
n_epochs = 100
n_fold = 3
include_chromatin_data = True

shuffle = StratifiedShuffleSplit(n_splits=n_fold,
                                 train_size = 0.7,
                                 test_size = 0.3,
                                 random_state = seed)

models2 = []
fithists2 = []

for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):
    
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    chrom_train, chrom_test = np.squeeze(chromatin[train_ix]), np.squeeze(chromatin[test_ix])
    
    print("Training on fold %d..."%(ifold+1))
    
    model = create_chromatin()
    hist = model.fit([X_train,chrom_train], y_train,
                     class_weight = class_weights,
                     batch_size = 1000,
                     epochs = n_epochs,
                     verbose = 0,
                     validation_data=([X_test,chrom_test],y_test))
    
    models2.append(model)
    fithists2.append(hist)
    
    print("Done")

In [ ]:
for i in range(shuffle.get_n_splits()):
    
    fig, [[ax1,ax2],[ax3,ax4]] = plt.subplots(2,2, figsize=(8,6))

    loss_rate_plot(fithists2[i], ax1, legend=True, label="Fold %d"%(i+1))
    accuracy_rate_plot(fithists2[i], ax2, label="Fold %d"%(i+1))
    precision_rate_plot(fithists2[i], ax3, label="Fold %d"%(i+1))
    recall_rate_plot(fithists2[i], ax4, label="Fold %d"%(i+1))

    fig.tight_layout()
    
plt.show()

In [ ]:
for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):

    print("---------------")
    print("Fold %d:"%(ifold+1))
    
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    chrom_train, chrom_test = chromatin[train_ix], chromatin[test_ix]
    
    model = models2[ifold]
    
    y_test_pred = model.predict([X_test,chrom_test])
    
    y_test_pred = np.round(y_test_pred)

    ax = plot_confusion_matrix(y_test, y_test_pred, ['0','1'], title="Confusion Matrix - Fold %d\n(Class-Weighted Samples)"%(ifold+1))


The code below is similar to, but slightly modified from, the ROC plot above. It shows the same quantity: the ROC curves for all k models from the k-fold cross-validation, and the mean and standard deviation assembled from those models.

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html

from scipy import interp

fig, ax = plt.subplots(1,1,figsize=(8,8))

tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)

for ifold, (train_ix, test_ix) in enumerate(shuffle.split(X,y)):

    print("Working on fold %d..."%(ifold+1))
    
    X_train, X_test = X[train_ix], X[test_ix]
    y_train, y_test = y[train_ix], y[test_ix]
    chrom_train, chrom_test = chromatin[train_ix], chromatin[test_ix]
    
    model = models2[ifold]
    
    y_test_pred = model.predict([X_test,chrom_test]).ravel()
        
    fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
    tprs.append(interp(mean_fpr,fpr,tpr))
    tprs[-1][0] = 0.0
    roc_auc = auc(fpr,tpr)
    aucs.append(roc_auc)
    ax.plot(fpr,tpr, alpha=0.3)

    
mean_tpr = np.mean(tprs,axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b', alpha=0.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.2)

ax.plot([0,1],[0,1],'--k',lw=2)

ax.set_title('ROC Curve, 1D CNN With Chromatin\n(Class-Weighted Samples)')

print("Area under curve:")
print("Mean: %0.4f"%(mean_auc))
print("Std: %0.4f"%(std_auc))

plt.show()